<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/11_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 記憶體管理

In [ ]:
#####  1.  ######
# 每一次循環中，重新load一筆資料並一樣命名為df，Python 會在新資料賦值給 df 時，自動釋放舊資料的引用

#####  2.  ######
import pandas as pd
import gc

# 儲存每批處理結果的列表
result_list = []

for i in range(20):  # 假設有20次，每次100萬筆資料
    # 載入一筆100萬的資料
    df = pd.read_csv(f'path_to_data_file_{i}.csv')

    # 進行一些計算，得到 result_df
    result_df = some_processing_function(df)

    # 將結果儲存到結果列表中
    result_list.append(result_df)

    # 刪除已處理的資料並清理內存
    del df  # 刪除變數 df 的引用。Python的垃圾回收機制會自動釋放不再使用的變數所佔的記憶體空間，del指令可加快這過程。
    gc.collect()
    # 手動觸發Python的垃圾回收器，即使有的物件仍在等待被釋放，使用gc.collect()也能強制清理內存。這在處理大量資料時特別有效。
    print(f"Processed batch {i+1} and cleared memory.")

# 合併所有結果
final_result = pd.concat(result_list, ignore_index=True)
print("All batches processed and concatenated.")


## 2. 兩個 df merge，NA部分改填0

In [ ]:
# 使用 'key' 欄位進行合併
merged_df = pd.merge(df1, df2, on='key', how='outer')

# 將空值填充為 0
merged_df = merged_df.fillna(0)

## 3. merge兩個df，並取min(release_time)

In [ ]:
import pandas as pd

# 範例資料
df1 = pd.DataFrame({
    'col1': [1, 2, 3],
    'col2': ['A', 'B', 'C'],
    'col3': ['X', 'Y', 'Z'],
    'col4': [10, 20, 30],
    'col5': [100, 200, 300],
    'col6': [1000, 2000, 3000],
    'rule_1': [5, 15, 25]
})

df2 = pd.DataFrame({
    'col1': [1, 2, 4],
    'col2': ['A', 'B', 'D'],
    'col3': ['X', 'Y', 'W'],
    'col4': [8, 25, 35],
    'col5': [110, 210, 310],
    'col6': [1100, 2100, 3100],
    'rule_2': [7, 17, 27]
})

# 欄位列表
lst = ['col1', 'col2', 'col3']
# 使用 lst 來選取需要的欄位進行合併
combined_df = pd.merge(
    df1[lst + ['col4', 'rule1']],
    df2[lst + ['col4', 'rule2']],
    on=lst,
    how='outer',
    indicator=True
)


# 對於 _merge 為 'both' 的行，選取 col4 的最小值 --------------------------
## way 1:
merged_df['col4'] = merged_df.apply(
    lambda row: min(row['col4_x'], row['col4_y']) if row['_merge'] == 'both' else (row['col4_x'] if pd.notna(row['col4_x']) else row['col4_y']),
    axis=1
)
# way 2:
combined_df['col4'] = combined_df[['col4_x', 'col4_y']].min(axis=1)
# 在合併後的 DataFrame 中，取 col4 最小的行
final_df = combined_df.loc[combined_df.groupby(['col1', 'col2', 'col3'])['min_col4'].idxmin()]
# 僅保留指定的欄位
final_df = final_df[['col1', 'col2', 'col3', 'col4', 'rule_1', 'rule_2']]




# 只保留需要的欄位，並填充 rule_1 和 rule_2 的 NaN 值為 0
combined_df = merged_df[['col1', 'col2', 'col3', 'col4', 'rule_1', 'rule_2']].fillna({'rule_1': 0, 'rule_2': 0})

# 查看結果
print(combined_df)


## 4. rule check

In [ ]:
re1 = []
re2 = []
re3 = []

for group, sub_data in df.groupby('chart_name'):
  # 取每個lot下面 time最小的那筆資訊
  data = sub_data.sort_values(['lot', 'time']).drop_duplicates('lot')
  # rule 計算
  trigger1 = rule1(data, n=2)
  trigger2 = rule2(data, n=4)
  trigger3 = rule3(data)
  # record
  re1.append(trigger1)
  re2.append(trigger2)
  re3.append(trigger3)

#
re1_raw = pd.concat(re1)
re1_raw['rule1'] = '1'
re2_raw = pd.concat(re1)
re2_raw['rule_ooc'] = '1'
re3_raw = pd.concat(re1)
re3_raw['rule_trend'] = '1'


# 同 col1,col2,lot下，不同param只保留一筆
re1_df = re1_raw.drop_duplicates(subset=['col1','col2', 'lot']).reset_index(drop=True)
re2_df = re2_raw.drop_duplicates(subset=['col1','col2', 'lot']).reset_index(drop=True)
re3_df = re3_raw.drop_duplicates(subset=['col1','col2', 'lot']).reset_index(drop=True)


# merge re1_df/re2_df/re3_df 結果，以col1,col2,lot做merge(how='outer'),並且只留下time數值最小的那筆

* 上面的精鍊版本

In [ ]:
import pandas as pd

# 儲存結果的空列表
results = {
    're1': [],
    're2': [],
    're3': []
}

# 使用 groupby 並針對每個 chart_name 計算 rule
for group, sub_data in df.groupby('chart_name'):
    # 取每個 lot 下 time 最小的資料
    data = sub_data.sort_values(['lot', 'time']).drop_duplicates('lot')

    # 計算規則並儲存
    results['re1'].append(rule1(data, n=2))
    results['re2'].append(rule2(data, n=4))
    results['re3'].append(rule3(data))

# 合併結果
re1_df_raw = pd.concat(results['re1'], ignore_index=True)
re2_df_raw = pd.concat(results['re2'])
re3_df_raw = pd.concat(results['re3'])


# merge所有